# Q&A System


In [135]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [136]:
from crewai import Agent, Task, Crew

In [137]:
import os
from utils import get_openai_api_key

openai_api_key = "sk-None-7dEvXzQXo1Kv05A70d7AT3BlbkFJmcLu8awojvh5tIlEDOrc"
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Role Playing

In [138]:
research_agent = Agent(
    role="Research Agent",
    goal="Gather and synthesize information from the Wikipedia page on Bangladesh.",
    backstory=(
        "This agent is responsible for thoroughly exploring and extracting relevant information from the Wikipedia page on Bangladesh. "
        "It compiles detailed data and ensures the accuracy and comprehensiveness of the information gathered. "
        "The purpose of this agent is to provide a solid foundation of knowledge for the Answer Agent to utilize in responding to user queries."
    ),
    allow_delegation=False,
    verbose=True,
)

In [139]:
answer_agent = Agent(
    role="Answer Agent",
    goal="Provide accurate and comprehensive answers to user questions using information from the Research Agent."
,
    backstory=(
        "This agent uses the synthesized information provided by the Research Agent to deliver detailed and factual answers to user questions about Bangladesh. "
        "It ensures that responses are complete, accurate, and based on verified content from the Wikipedia page. "
        "The Answer Agent's purpose is to facilitate learning and knowledge dissemination by leveraging the reliable information gathered by the Research Agent."
    ),
    verbose=True,
)

## Custom tool for WebScrap

In [140]:
from crewai_tools import SerperDevTool,ScrapeWebsiteTool, WebsiteSearchTool

In [141]:
from crewai_tools import BaseTool

In [142]:
import requests
from bs4 import BeautifulSoup
import json

    
def para(soup):
    paragraphs = soup.find_all('p')
    paragraph_texts = [para.get_text(strip=True) for para in paragraphs]
    all_paragraphs = '\n'.join(paragraph_texts)
    return all_paragraphs


def extract_all_tables(soup):
    tables = soup.find_all('table')
    all_tables_data = []
    for table_index, table in enumerate(tables):
        headers = [th.get_text(strip=True) for th in table.find_all('th')]
        data = []
        for row in table.find_all('tr'):
            cols = row.find_all(['th', 'td'])
            cols_text = [col.get_text(strip=True) for col in cols]
            if cols_text:
                data.append(cols_text)
        table_str = f"Table {table_index + 1}:\n"
        if headers:
            table_str += " | ".join(headers) + "\n"
        for row in data:
            table_str += " | ".join(row) + "\n"
        
        all_tables_data.append(table_str.strip()) 

    all_tables_data=all_tables_data[:4]
    return "\n".join(all_tables_data)

def scrape_wikipedia_page():
    url = 'https://en.wikipedia.org/wiki/Bangladesh'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraph=para(soup)
    tables=extract_all_tables(soup)
    Info=paragraph[:len(paragraph)//2] +'\n'+tables
    return Info

In [143]:
text=scrape_wikipedia_page()
print(len(text))

56026


In [144]:
class WebScrapTool(BaseTool):
    name: str ="Web Scrap Tool"
    description: str = ("Gather the data from Wikipedia "
         "to ensure data collection.")
    
    def _run(self, text: str) -> str:
        text=scrape_wikipedia_page()
        return text

In [145]:
webScrapTool=WebScrapTool()

### Tasks

In [146]:
inquiry_research = Task(
    description=(
        "{person} just reached out with an important question:\n"
        "{question}\n\n"
        "{name} asked a question. "
        "Research and gather detailed information from the Wikipedia page on Bangladesh to address this inquiry."
    ),
    expected_output=(
        "Detailed and accurate information about Bangladesh relevant to the inquiry."
    ),
    tools=[webScrapTool],
    agent=research_agent,
)

In [147]:
inquiry_answer = Task(
    description=(
        "Using the research data provided by the Research Agent, answer the following inquiry:\n"
        "{question}\n\n"
        "Ensure the answer is specific and accurate based on the information gathered."
        "use only the given information"
    ),
    expected_output=(
        "A specific and accurate answer based on the research data."
    ),
    agent=answer_agent,
)


In [148]:
crew = Crew(
    agents=[research_agent, answer_agent],
    tasks=[inquiry_research, inquiry_answer],
    verbose=2,
    memory=True
)

2024-08-01 02:22:46,606 - 139856631978880 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


### Running the Crew for Questions

In [149]:
inputs = {
    "person": "Rakib",
    "name": "Rakib",
    "question": "What is the total population of Chittagong?"
}

# Kickoff the crew
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Research Agent
 [INFO]: == Starting Task: Rakib just reached out with an important question:
What is the total population of Chittagong?

Rakib asked a question. Research and gather detailed information from the Wikipedia page on Bangladesh to address this inquiry.


> Entering new CrewAgentExecutor chain...
I should start by extracting the relevant information about the population of Chittagong from the Wikipedia page on Bangladesh.

Action: Web Scrap Tool
Action Input: {"text": "population of Chittagong Bangladesh Wikipedia"} 


Bangladesh,[a]officially thePeople's Republic of Bangladesh,[b]is a country inSouth Asia. It is theeighth-most populouscountry in the world and is among themost densely populated countrieswith a population of nearly 170 million in an area of 148,460 square kilometres (57,320 sq mi). Bangladesh shares land borders with India to the north, west, and east, andMyanmarto the southeast. To the south, it has a coastline along theBay of Be

Final Answer: 
The total population of Chittagong, the second-largest city in Bangladesh, is 3,227,246 according to the latest data available. Chittagong is a significant port city on the Bay of Bengal and plays a crucial role in the economic activities of the country.

> Finished chain.
 [DEBUG]: == [Research Agent] Task output: The total population of Chittagong, the second-largest city in Bangladesh, is 3,227,246 according to the latest data available. Chittagong is a significant port city on the Bay of Bengal and plays a crucial role in the economic activities of the country.


 [DEBUG]: == Working Agent: Answer Agent
 [INFO]: == Starting Task: Using the research data provided by the Research Agent, answer the following inquiry:
What is the total population of Chittagong?

Ensure the answer is specific and accurate based on the information gathered.use only the given information


> Entering new CrewAgentExecutor chain...
I need to provide the total population of Chittagong based o

In [150]:
from IPython.display import Markdown
Markdown(result)

The total population of Chittagong, the second-largest city in Bangladesh, is 3,227,246 according to the latest data available.

In [151]:
inputs = {
    "person": "Rakib",
    "name": "Rakib",
    "question": "Who is the President of Bangladesh?"
}

# Kickoff the crew
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Research Agent
 [INFO]: == Starting Task: Rakib just reached out with an important question:
Who is the President of Bangladesh?

Rakib asked a question. Research and gather detailed information from the Wikipedia page on Bangladesh to address this inquiry.


> Entering new CrewAgentExecutor chain...
I should start by extracting the relevant information about the President of Bangladesh from the Wikipedia page on Bangladesh.

Action: Web Scrap Tool
Action Input: {"text": "President of Bangladesh Wikipedia"} 


Bangladesh,[a]officially thePeople's Republic of Bangladesh,[b]is a country inSouth Asia. It is theeighth-most populouscountry in the world and is among themost densely populated countrieswith a population of nearly 170 million in an area of 148,460 square kilometres (57,320 sq mi). Bangladesh shares land borders with India to the north, west, and east, andMyanmarto the southeast. To the south, it has a coastline along theBay of Bengal. It is narrowly 

Final Answer: The President of Bangladesh is Mohammed Shahabuddin.

> Finished chain.
 [DEBUG]: == [Research Agent] Task output: The President of Bangladesh is Mohammed Shahabuddin.


 [DEBUG]: == Working Agent: Answer Agent
 [INFO]: == Starting Task: Using the research data provided by the Research Agent, answer the following inquiry:
Who is the President of Bangladesh?

Ensure the answer is specific and accurate based on the information gathered.use only the given information


> Entering new CrewAgentExecutor chain...
I need to provide the accurate and specific answer to the question of who is the President of Bangladesh based on the information provided by the Research Agent.

Action: Ask question to co-worker
Action Input: {"coworker": "Research Agent", "question": "Who is the President of Bangladesh?", "context": "The President of Bangladesh is Mohammed Shahabuddin."}


> Entering new CrewAgentExecutor chain...
I now can give a great answer.

Final Answer: 
The President of Bangl

- Display the final result as Markdown.

In [152]:
from IPython.display import Markdown
Markdown(result)

The President of Bangladesh is Mohammed Shahabuddin.